In [1]:
# Imports
import inspect
import random
import tempfile
import uuid
from functools import wraps
import collections

import coloredlogs
import dask as dd
import logging
import numpy as np
import pandas as pd
from coolname import generate_slug

# Logger
logger = logging.getLogger(__name__)
coloredlogs.install(level='DEBUG')

# StackOverflow snippet #1
########################################################################################################################
# /////|   Decorator   |////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################


def auto_assign_arguments(function):

    @wraps(function)
    def wrapped(self, *args, **kwargs):
        _assign_args(self, list(args), kwargs, function)
        function(self, *args, **kwargs)

    return wrapped


########################################################################################################################
# /////|   Utils   |////////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################


def _assign_args(instance, args, kwargs, function):

    def set_attribute(instance, parameter, default_arg):
        if not (parameter.startswith("_")):
            setattr(instance, parameter, default_arg)

    def assign_keyword_defaults(parameters, defaults):
        for parameter, default_arg in zip(reversed(parameters),
                                          reversed(defaults)):
            set_attribute(instance, parameter, default_arg)

    def assign_positional_args(parameters, args):
        for parameter, arg in zip(parameters, args.copy()):
            set_attribute(instance, parameter, arg)
            args.remove(arg)

    def assign_keyword_args(kwargs):
        for parameter, arg in kwargs.items():
            set_attribute(instance, parameter, arg)

    def assign_keyword_only_defaults(defaults):
        return assign_keyword_args(defaults)

    def assign_variable_args(parameter, args):
        set_attribute(instance, parameter, args)

    POSITIONAL_PARAMS, VARIABLE_PARAM, _, KEYWORD_DEFAULTS, _, KEYWORD_ONLY_DEFAULTS, _ = inspect.getfullargspec(
        function)
    POSITIONAL_PARAMS = POSITIONAL_PARAMS[1:]  # remove 'self'

    if (KEYWORD_DEFAULTS):
        assign_keyword_defaults(parameters=POSITIONAL_PARAMS,
                                defaults=KEYWORD_DEFAULTS)
    if (KEYWORD_ONLY_DEFAULTS):
        assign_keyword_only_defaults(defaults=KEYWORD_ONLY_DEFAULTS)
    if (args): assign_positional_args(parameters=POSITIONAL_PARAMS, args=args)
    if (kwargs): assign_keyword_args(kwargs=kwargs)
    if (VARIABLE_PARAM):
        assign_variable_args(parameter=VARIABLE_PARAM, args=args)


########################################################################################################################


########################################################################################################################
# /////|   Class   |////////////////////////////////////////////////////////////////////////////////////////////////////
########################################################################################################################
class initSim:
    """Class to initialize the progenitor (F0) population.

    initSim primarily creates an object which stores information about the 
    initial population. Moreover the inbuilt methods will return pandas
    dataframe for direct modification as desired.
    
    :return: Object
    :rtype: object type initSim
    """

    @auto_assign_arguments
    def __init__(self,
                 tcount=1,
                 ttype=[1],
                 popsize=100,
                 insize=2000,
                 gensize=1000,
                 trate=[0.02],
                 tpenalty=[-0.02],
                 tparent=["Mother"]):
        """Class constructor
        :param tcount: Number of transposons to be present in initial population
        :param ttype: Type of transposons (class 1 or 2)
        :param popsize: Population size
        :param insize: Number of insertion sites
        :param gensize: Size of genome on cM
        :param trate: Transposition rates
        :param tpenalty: Selection penalty for each transposon
        :param tparent: Parent carrying the transposon insertion (Mother/Father)
        :type tcount: int
        :type ttype: list [int]
        :type popsize: int
        :type insize: int
        :type gensize: int
        :type trate: list [float]
        :type tpenalty: list [float]
        :type arg1: tparent [string]
        """
        # Dataframe definations
        self.TranspFrame = pd.DataFrame(columns=[
            'TID', 'Position', 'SelCo', 'Name', 'Class', 'Traceback',
            'Generation', 'Parent', "TraRate"
        ])
        self.PopFrame = pd.DataFrame(columns=[
            'PID', 'Fitness', 'Name', 'Sex', 'Lineage', 'Generation',
            'TEfather', 'TEmother', 'Insertion_Father', 'Insertion_Mother'
        ])

    # Init transposons
    def initT(self):
        """Method to dataframe containing initial transposon population
        
        :return: Dataframe containing Transposon information
        :rtype: Dataframe
        """

        if (self.tcount > len(self.tpenalty)):
            logger.info(
                "Mismatch between transposon count and selection penalties. Using default for each transposon count!"
            )
            self.tpenalty = [-0.4] * self.tcount
        if (self.tcount > len(self.trate)):
            logger.info(
                "Mismatch between transposon count and transposition rates. Using default for each transposon count!"
            )
            self.trate = [0.02] * self.tcount
        if (self.tcount > len(self.ttype)):
            logger.info(
                "Mismatch between transposon count and transposon types. Using default for each transposon count!"
            )
            self.ttype = [1] * self.tcount
        if (self.tcount > len(self.tparent)):
            logger.info(
                "Mismatch between transposon count and transposon parent. Using default for each transposon count!"
            )
            self.tparent = ["Mother"] * self.tcount

        # Create random filled insertion sites
        inSiteArray = random.sample(range(1, self.insize), self.tcount)

        for i in range(0, self.tcount):
            row = pd.Series({
                'TID': uuid.uuid4().hex,
                'Position': inSiteArray[i],
                'TraRate': self.trate[i],
                'SelCo': self.tpenalty[i],
                'Name': generate_slug(),
                'Class': self.ttype[i],
                'Traceback': ['0'],
                'Generation': 1,
                'Parent': self.tparent[i]
            })
            self.TranspFrame = self.TranspFrame.append(row, ignore_index=True)
        return (self.TranspFrame)

    # Init population and genome
    def initPG(self):
        """Method to create initial population and their respective genomes
        
        :return: tuple(population,genome)
            WHERE
            population is population dataframe
            genome is genome dataframe
        :rtype: DataFrame
        """

        # Create transposon insertions in randomly selected individuals
        IndividualToInsert = random.sample(list(range(1, self.popsize)),
                                           self.tcount)
        TIDlist = self.TranspFrame.TID.tolist()
        TIDcounter = 0
        Parent = "0"
        insertion_Father = 0
        insertion_Mother = 0
        FitnessPen = 0
        TEfather = "0"
        TEmother = "0"
        for i in range(self.popsize):
            # In case this (un)lucky individual has transposon insertion
            if (i in IndividualToInsert):
                TE = TIDlist[TIDcounter]
                TIDcounter += 1
                Parent = self.TranspFrame[self.TranspFrame['TID'] ==
                                          TE]['Parent'].values[0]
                FitnessPen = self.TranspFrame[self.TranspFrame['TID'] ==
                                              TE]['SelCo'].values[0]
                if (Parent == "Mother"):
                    insertion_Mother = self.TranspFrame[
                        self.TranspFrame['TID'] == TE]['Position'].values[0]
                    TEmother = TE

                if (Parent == "Father"):
                    insertion_Father = self.TranspFrame[
                        self.TranspFrame['TID'] == TE]['Position'].values[0]
                    TEfather = TE

            else:
                TE = '0'
                Parent = "0"
                insertion_Father = 0
                insertion_Mother = 0
                FitnessPen = 0
                TEmother = "0"
                TEfather = "0"

            # Populate the population!
            rowPop = pd.Series({
                'PID': uuid.uuid4().hex,
                'Fitness': random.uniform(0.6, 1.0) - FitnessPen,
                'Name': generate_slug(),
                'Sex': 'H',
                'Lineage': ['0'],
                'Generation': 1,
                'Insertion_Father': [insertion_Father],
                'Insertion_Mother': [insertion_Mother],
                'TEmother': [TEmother],
                'TEfather': [TEfather]
            })
            self.PopFrame = self.PopFrame.append(rowPop, ignore_index=True)

        self.PopFrame['Lineage'] = self.PopFrame['Lineage'].astype('object')
        self.PopFrame['Insertion_Father'] = self.PopFrame[
            'Insertion_Father'].astype('object')
        self.PopFrame['Insertion_Mother'] = self.PopFrame[
            'Insertion_Mother'].astype('object')
        return (self.PopFrame)

    def createSim(self):
        """Method to generate the initial simulation dataset
        
        :return: List
            WHERE 
            index 0 is transposon dataframe
            index 1 is population dataframe
            index 2 is genome dataframe
        :rtype: list
        """

        transposon = self.initT()
        genome = self.initPG()
        return ([transposon, genome])
########################################################################################################################

Genome = pd.DataFrame(
    columns=['Position', 'RecombinationRate', 'InsertionSite'])


def generateGenome(genomeSize, nInsertionSites, nChrom):
    genome = list(range(1, genomeSize + 1))
    InsertionSite = [0] * len(genome)
    RecombinationRate = [0.01] * len(genome)
    insertionLocation = np.random.choice(genomeSize,
                                         nInsertionSites,
                                         replace=False)
    chrLocation = np.random.choice(genomeSize, nChrom, replace=False)

    insertionCounter = 1
    for i in insertionLocation:
        InsertionSite[i] = insertionCounter
        insertionCounter += 1

    insertionCounter = 1
    for i in chrLocation:
        if (InsertionSite[i] != 0):
            RecombinationRate[i + 1] = 0.5
            insertionCounter += 1
        else:
            RecombinationRate[i] = 0.5
            insertionCounter += 1

    genomeDict = {
        'Position': genome,
        'RecombinationRate': RecombinationRate,
        'InsertionSite': InsertionSite
    }
    genome = pd.DataFrame(genomeDict)
    return (genome)

In [32]:
def recombination(genomeFrame, genomeOrg):
    # Create insertion list for the progeny
    TEprogeny = []
    TEid = []
    # Create a copy of genomeFrame
    genomeCopy = genomeFrame.copy(deep=True)
    TEid_Father = genomeOrg['TEfather']
    TEid_Mother = genomeOrg['TEmother']
    Insertion_Father = genomeOrg['Insertion_Father']
    Insertion_Mother = genomeOrg['Insertion_Mother']
    if (Insertion_Father[0] == 0 and Insertion_Mother[0] == 0):
        return ([0],
                random.choice([
                    genomeOrg['Insertion_Father'], genomeOrg['Insertion_Mother']
                ]))
    else:
        initParent = random.choice(["M", "F"])
        surrogateParent = "M" if ("M" != initParent) else "F"
        RandArray = np.random.uniform(0, 1.0, genomeFrame.shape[0])
        switch = genomeCopy['RecombinationRate'] > RandArray
        #counter = collections.Counter(switch)
        #print(counter)
        genomeCopy['Progeny'] = np.where(switch.cumsum() % 2 == 0, initParent,
                                         surrogateParent)
        if all(v == 0 for v in Insertion_Father):
            pass
        else:
            for i in Insertion_Father:
                se = genomeCopy[genomeCopy['InsertionSite'] ==
                                i]['Progeny'].values[0]
                if (se == "M"):
                    TEprogeny.append(i)
                    TEid.append(TEid_Father.pop(0))
        if all(v == 0 for v in Insertion_Mother):
            pass
        else:
            for i in Insertion_Mother:
                se = genomeCopy[genomeCopy['InsertionSite'] ==
                                i]['Progeny'].values[0]
                if (se == "F"):
                    TEprogeny.append(i)
                    TEid.append(TEid_Mother.pop(0))
        #print(genomeCopy['Progeny'].value_counts())
    return (TEprogeny, TEid)

In [33]:
genome = generateGenome(100000, 2000, 100)

In [34]:
k = initSim(tcount=80)
t = k.createSim()
t[1]

2019-12-03 16:48:14 siddharth-OptiPlex-7070 __main__[4949] INFO Mismatch between transposon count and selection penalties. Using default for each transposon count!
2019-12-03 16:48:14 siddharth-OptiPlex-7070 __main__[4949] INFO Mismatch between transposon count and transposition rates. Using default for each transposon count!
2019-12-03 16:48:14 siddharth-OptiPlex-7070 __main__[4949] INFO Mismatch between transposon count and transposon types. Using default for each transposon count!
2019-12-03 16:48:14 siddharth-OptiPlex-7070 __main__[4949] INFO Mismatch between transposon count and transposon parent. Using default for each transposon count!


,PID,Fitness,Name,Sex,Lineage,Generation,TEfather,TEmother,Insertion_Father,Insertion_Mother
0,9cb12b7a467d4b13aa95fa74871a4cac,0.622083,evasive-magnetic-frog-of-freedom,H,[0],1,[0],[0],[0],[0]
1,2ae7fb6d3d34444a87122603265fa0e8,1.182753,watchful-toad-of-ancient-attraction,H,[0],1,[0],[f9292d431aa34db2901c35b4d9eef1cd],[0],[184]
2,97fd554cc023479b8f3521e15246f8bc,1.355970,fuzzy-leech-of-amazing-examination,H,[0],1,[0],[ca458f4b27724f74baf98626279232d0],[0],[115]
3,9aa17104f494425d91caa5074d276809,0.877984,brave-savvy-guan,H,[0],1,[0],[0],[0],[0]
4,abbbebfe1935422c822fd15c05d1fc02,1.377646,macho-light-oyster-of-completion,H,[0],1,[0],[d31ea11886b6443387d3ac8c09ed2231],[0],[403]
...,...,...,...,...,...,...,...,...,...,...
95,d46b95e7829740238e936b2f67f77645,1.094802,outrageous-horse-of-wondrous-passion,H,[0],1,[0],[2c912bbaee344c789da54857b069cccd],[0],[1734]
96,62fbcb66fa924f1ab361d0e9ba544bf9,1.022472,passionate-bright-shark-of-glory,H,[0],1,[0],[aba2dbcdd9a04e62a9c7c9a0fe981e65],[0],[1746]
97,65dac47292dd4627ae16ef5f5b67f8e5,1.142726,lively-mini-millipede-of-infinity,H,[0],1,[0],[7d60ccb660254058af94da0ba3c7c495],[0],[1303]
98,6613cce6fa5c4e84883e63e6596fab07,1.266076,optimistic-nimble-hedgehog-of-joy,H,[0],1,[0],[4c59f54cd2e847ae840b1b47d1b3e03c],[0],[1229]


In [35]:
for i in (range(t[1].shape[0])):
    print(recombination(genome, t[1].iloc[i].copy(deep=True)))

([0], [0])
([184], ['f9292d431aa34db2901c35b4d9eef1cd'])
([115], ['ca458f4b27724f74baf98626279232d0'])
([0], [0])
([403], ['d31ea11886b6443387d3ac8c09ed2231'])
([], [])
([], [])
([], [])
([655], ['5dbf93c37133432594514c92861f11af'])
([0], [0])
([0], [0])
([], [])
([789], ['cd21894184cc4314b0e876182461655f'])
([269], ['296548dedb124fee9ec19ed80a2a8ae7'])
([], [])
([0], [0])
([0], [0])
([], [])
([], [])
([456], ['b2564d1173894823bd44c113701ec2a5'])
([948], ['876b9c7d34bf487686a902de7434cb73'])
([], [])
([], [])
([], [])
([0], [0])
([1406], ['e25a1b10f8ba4397aa846bcefcc8397d'])
([], [])
([], [])
([], [])
([], [])
([], [])
([], [])
([], [])
([1715], ['84d15df1022a45a59adf5af5549fb372'])
([161], ['f3283a34ba564ac387f3e46316717afe'])
([1538], ['7a6b6c05352a488397bc8e20a6051406'])
([], [])
([], [])
([0], [0])
([335], ['8d602e85629943618041d0fde89d2862'])
([], [])
([1257], ['6f7b0ceb28fb4a00b97614ac48e953e9'])
([], [])
([1463], ['1011aa078a1143b2ab278f6b3e24447d'])
([0], [0])
([], [])
([803], 

M
F
